In [1]:
from model.cnn import CNN
import tensorflow as tf

2022-12-11 00:37:51.340258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 00:37:53.356847: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-11 00:37:53.356930: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-11 00:37:53.356940: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
from model.utils import Params


    # compile model. This initializes the variables.

params = Params("./model/params.json")
ckpt = "./experiments/saved_weights/"
model = CNN(num_classes=params.num_labels, checkpoint_directory=ckpt, params=params)
# model.build((1,224,224,3))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=params.learning_rate),)

2022-12-11 00:38:35.354276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:38:35.443776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:38:35.444108: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-11 00:38:35.445695: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
model.load_weights("./experiments/saved_weights/weight")

In [4]:
from model.input_fn import _parse_function

x = _parse_function("../data/224x224_currency/train_dir/2000_IMG20221111124053.jpg",1,224)
print(x[0].shape)


(224, 224, 3)


In [5]:
import numpy as np
arr = np.zeros((1,224,224,3),dtype=np.float32)
print(arr.shape)

(1, 224, 224, 3)


In [6]:
# model.device = "gpu:0"
arr[0] = x[0]
# model.build((1, 224, 224, 3))
model.summary()

Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  896       
                                                                 
 conv2d_1 (Conv2D)           multiple                  9248      
                                                                 
 conv2d_2 (Conv2D)           multiple                  25632     
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           multiple                  0 (unuse

In [7]:
arr[0] = x[0]
logits = model.call(arr, training=True)
prob = tf.nn.softmax(logits, axis=1)
pred = tf.argmax(prob, axis=1)
print(pred)

2022-12-11 00:38:45.680025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600


tf.Tensor([6], shape=(1,), dtype=int64)


In [8]:
x1 = np.array([6])
model.fit(arr,x1)
model.save("./custom2")

2022-12-11 00:38:51.411272: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape incnn/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2022-12-11 00:38:51.850643: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f241f2962a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-11 00:38:51.850692: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2022-12-11 00:38:51.875232: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-12-11 00:38:52.148969: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 5s 5s/step - loss: 5.3590


INFO:tensorflow:Assets written to: ./custom2/assets


INFO:tensorflow:Assets written to: ./custom2/assets


In [9]:
# Saving the model
# model.build((1,224,224,3))




# Saving tflite model
from model.tfliteconvert import convert_to_tflite
convert_to_tflite("./custom2")

2022-11-13 13:49:35.676494: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-13 13:49:35.676548: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-13 13:49:35.682020: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./custom2
2022-11-13 13:49:35.686574: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-13 13:49:35.687133: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ./custom2
2022-11-13 13:49:35.700585: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-13 13:49:35.705869: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-13 13:49:35.906665: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: ./custom2
2022-11-13 13:49:35.941544: 

In [13]:
# model.build((1,224,224,3))
model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=params.learning_rate),
)
# model.call(arr,training=False)



1/1 [==============================] - 3s 3s/step - loss: 51.0227


INFO:tensorflow:Assets written to: ./custom2/assets


INFO:tensorflow:Assets written to: ./custom2/assets


In [10]:
interpreter = tf.lite.Interpreter("./model2.tflite")
interpreter.allocate_tensors()

In [11]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.set_tensor(input_details[0]['index'], arr)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
output_data

array([[  8.77212  ,   4.579085 ,   2.7976358,  10.501842 ,  12.886151 ,
        -19.530235 ,  36.973637 ]], dtype=float32)

In [12]:
results = np.squeeze(output_data)
results

array([  8.77212  ,   4.579085 ,   2.7976358,  10.501842 ,  12.886151 ,
       -19.530235 ,  36.973637 ], dtype=float32)

In [ ]:
top_k = results.argsort()[-5:][::-1]
top_k

In [3]:
# Evaluate TFLite Model
import os
correct = 0
incorrect = 0
files = os.listdir("../data/test_dir")

    

<class 'list'>


In [3]:
from model.cnn import CNN
from model.utils import Params
import tensorflow as tf

    # compile model. This initializes the variables.

params = Params("./model/params.json")
ckpt = "./experiments/saved_weights/"
model = CNN(num_classes=params.num_labels, checkpoint_directory=ckpt, params=params)
# model.build((1,224,224,3))
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer=tf.keras.optimizers.Adam(learning_rate=params.learning_rate),)

In [12]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore("./experiments/checkpoints/ckpt-80")

In [17]:
from model.input_fn import _parse_function

x = _parse_function("../data/224x224_currency/train_dir/500_IMG-20220928-WA0036.jpg",1,224)

In [18]:
import numpy as np
arr = np.zeros((1,224,224,3),dtype=np.float32)
arr[0] = x[0]
logits = model.call(arr, training=True)
prob = tf.nn.softmax(logits, axis=1)
pred = tf.argmax(prob, axis=1)
print(pred)

tf.Tensor([5], shape=(1,), dtype=int64)
